In [9]:
# notebook_1.ipynb
# Web3 Trading Team Data Science Assignment

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

TRADER_DATA_PATH = 'csv_files/historical_data.csv'
SENTIMENT_DATA_PATH = 'csv_files/fear_greed_index.csv'

# --- 1. Read CSV files ---
trader_df = pd.read_csv(TRADER_DATA_PATH)
trader_df.columns = [col.strip() for col in trader_df.columns]

# For the sentiment data: whitespace-separated columns, fix the header (no commas!)
sentiment_df = pd.read_csv(
    SENTIMENT_DATA_PATH,
    delim_whitespace=True,
    header=None,
    names=['timestamp', 'value', 'Classification', 'Date']
)
sentiment_df.columns = [col.strip() for col in sentiment_df.columns]

print('\n== trader_df columns ==\n', trader_df.columns.tolist())
print('\n== sentiment_df columns ==\n', sentiment_df.columns.tolist())

# --- 2. Convert date columns ---
if 'Timestamp IST' in trader_df.columns:
    trader_df['Timestamp IST'] = pd.to_datetime(trader_df['Timestamp IST'], errors='coerce')
    trader_df['date_only'] = trader_df['Timestamp IST'].dt.strftime('%Y-%m-%d')
else:
    print("'Timestamp IST' not found in trader_df columns.")

if 'Date' in sentiment_df.columns:
    sentiment_df['Date'] = pd.to_datetime(sentiment_df['Date'], errors='coerce')
    sentiment_df['date_only'] = sentiment_df['Date'].dt.strftime('%Y-%m-%d')
else:
    print("'Date' not found in sentiment_df columns.")

# Show unique date values to debug join/overlap issues
print("\nUnique date_only values in trader_df:", trader_df['date_only'].dropna().unique()[:10])
print("Unique date_only values in sentiment_df:", sentiment_df['date_only'].dropna().unique()[:10])

# --- 3. Merge dataframes safely (both sides now have string date_only) ---
if 'date_only' in sentiment_df.columns and 'Classification' in sentiment_df.columns:
    merged_df = trader_df.merge(
        sentiment_df[['date_only', 'Classification']],
        on='date_only',
        how='left'
    )
    print('\n== Merge successful ==')
else:
    raise KeyError("Expected columns 'date_only' and 'Classification' not found in sentiment_df. Please check your CSV format/content.")

# --- Output top rows for verification ---
print("Merged Data Sample:\n", merged_df[['date_only', 'Classification']].head(20))

# --- 4. Ensure outputs directory exists ---
os.makedirs('outputs', exist_ok=True)

# --- 5. Save merged data ---
merged_df.to_csv('csv_files/merged_trader_sentiment.csv', index=False)

# --- 6. Plots ---
if merged_df['Classification'].notna().sum() == 0:
    print("No sentiment classification data matched with trader data! All 'Classification' values are NaN. Check date ranges and formats.")
else:
    plt.figure(figsize=(8, 4))
    sns.countplot(x='Classification', data=merged_df)
    plt.title('Number of Trades During Fear vs Greed')
    plt.savefig('outputs/trade_count_fear_greed.png')
    plt.show()

if 'Closed PnL' in merged_df.columns and merged_df['Classification'].notna().sum() > 0:
    plt.figure(figsize=(8, 4))
    sns.boxplot(x='Classification', y='Closed PnL', data=merged_df)
    plt.title('Profit & Loss Distribution by Sentiment')
    plt.savefig('outputs/profit_dist_fear_greed.png')
    plt.show()
elif 'Closed PnL' not in merged_df.columns:
    print("'Closed PnL' not found in merged_df columns.")
else:
    print("No sentiment classification data for PnL plot. All 'Classification' values are NaN.")

# --- END OF notebook_1.ipynb ---



== trader_df columns ==
 ['Account', 'Coin', 'Execution Price', 'Size Tokens', 'Size USD', 'Side', 'Timestamp IST', 'Start Position', 'Direction', 'Closed PnL', 'Transaction Hash', 'Order ID', 'Crossed', 'Fee', 'Trade ID', 'Timestamp']

== sentiment_df columns ==
 ['timestamp', 'value', 'Classification', 'Date']

Unique date_only values in trader_df: ['2024-02-12' '2024-03-12' '2024-04-12' '2025-04-03' '2025-05-03'
 '2025-11-03' '2025-03-18' '2025-08-04' '2025-09-04' '2025-10-04']
Unique date_only values in sentiment_df: ['NaT']

== Merge successful ==
Merged Data Sample:
      date_only  Classification
0   2024-02-12             NaN
1   2024-02-12             NaN
2   2024-02-12             NaN
3   2024-02-12             NaN
4   2024-02-12             NaN
5   2024-02-12             NaN
6   2024-02-12             NaN
7   2024-02-12             NaN
8   2024-02-12             NaN
9   2024-02-12             NaN
10  2024-02-12             NaN
11  2024-02-12             NaN
12  2024-02-12  